In [5]:
import numpy as np
import math
from shapely.geometry import Point,Polygon
import matplotlib.pyplot as plt
from ca_utils import *

In [6]:
def normalize(vector):      # return: The vector scaled to a length of 1
    norm = math.sqrt(vector[0] ** 2 + vector[1] ** 2)
    return vector[0] / norm, vector[1] / norm

def dot(vector1, vector2):  # return: The dot (or scalar) product of the two vectors
    return vector1[0] * vector2[0] + vector1[1] * vector2[1]

def edge_direction(point0: Point, point1: Point):   # return: A vector going from point0 to point1
    return point1.x - point0.x, point1.y - point0.y

def orthogonal(vector):     # return: A new vector which is orthogonal to the given vector
    return vector[1], -vector[0]

def vertices_to_edges(vertices):    # return: A list of the edges of the vertices as vectors
    return [edge_direction(vertices[i], vertices[(i + 1) % len(vertices)])for i in range(len(vertices))]

def project(vertices, axis):    # return: A vector showing how much of the vertices lies along the axis
    dots = [dot(vertex, axis) for vertex in vertices]
    return [min(dots), max(dots)]

def interval_intersection(intervals: list):
    p = [-float("inf"),float("inf")]
    for i in range(len(intervals)):
        if (min(intervals[i][1],p[1]) - max(intervals[i][0],p[0])) > 0:
            p = [max(intervals[i][0],p[0]),min(intervals[i][1],p[1])]
        else:
            return [-float("inf"),float("inf")]
    return p
    
def local_overlapping_interval(p1:list, p2: list, pv1: float, pv2:float):
    if (pv1 - pv2) != 0:
        s0x = [((p2[0]-p1[0])/(pv1-pv2)),((p2[1]-p1[0])/(pv1-pv2))]
        s1x = [((p1[0]-p2[0])/(pv2-pv1)),((p1[1]-p2[0])/(pv2-pv1))]
        return [min(s0x+s1x),max(s0x+s1x)]
    else:
        return [-float("inf"),float("inf")]

def separating_axis_theorem(obj_a: moving_object, obj_b: moving_object):
    vertices_a = obj_a.vertices
    vertices_b = obj_b.vertices

    va_x = round(obj_a.speed*math.sin(obj_a.heading),5)
    va_y = round(obj_a.speed*math.cos(obj_a.heading),5)

    vb_x = round(obj_b.speed*math.sin(obj_b.heading),5)
    vb_y = round(obj_b.speed*math.cos(obj_b.heading),5)

    edges = vertices_to_edges(vertices_a) + vertices_to_edges(vertices_b)
    axes = [normalize(orthogonal(edge)) for edge in edges]

    overlapping_intervals = []

    for axis in axes:
        projection_a = project(vertices_a, axis)
        projection_b = project(vertices_b, axis)
        speed_projection_a = dot([va_x,va_y],axis)
        speed_projection_b = dot([vb_x,vb_y],axis)
        loi = local_overlapping_interval(projection_a,projection_b,speed_projection_a,speed_projection_b)
        overlapping_intervals.append(loi)

    collision_interval = interval_intersection(overlapping_intervals)

    return collision_interval

[-inf, inf]